In [7]:
import pandas as pd
import random
from datetime import datetime
from dateutil.relativedelta import relativedelta

product_df = pd.read_csv(
    '/Users/ren/Documents/workspace/E-Commerce-Analysis/data/ecommerce-data/data/olist-ecommerce/olist_products_dataset.csv')
clickstream_df = pd.read_csv(
    '/Users/ren/Documents/workspace/E-Commerce-Analysis/data/raw/ecommerce_clickstream_transactions.csv')

product_ids = product_df['product_id'].tolist()

product_id_clickstream = clickstream_df['ProductID'].unique()
product_id_clickstream = [
    pid for pid in product_id_clickstream if pd.notna(pid)]

mapping = {pid: random.choice(product_ids) for pid in product_id_clickstream}

clickstream_df['ProductID'] = clickstream_df['ProductID'].map(
    lambda x: mapping.get(x, x))

clickstream_df['Timestamp'] = pd.to_datetime(
    clickstream_df['Timestamp'], errors='coerce')


def shift_year(timestamp):
    if pd.isna(timestamp):
        return timestamp
    new_year = random.choice([2017, 2018])
    try:
        return timestamp.replace(year=new_year)
    except ValueError:
        while True:
            try:
                return timestamp.replace(year=new_year, day=timestamp.day - 1)
            except ValueError:
                continue

clickstream_df['Timestamp'] = clickstream_df['Timestamp'].apply(shift_year)

clickstream_df = clickstream_df.sort_values(by='Timestamp')

clickstream_df.to_csv(
    '/Users/ren/Documents/workspace/E-Commerce-Analysis/data/updated/ecommerce_clickstream_transactions_updated.csv', index=False)

In [10]:
import pandas as pd

df = pd.read_csv(
    '../data/updated/ecommerce_clickstream_transactions_updated.csv')


def format_log_entry(row):
    timestamp = row['Timestamp']
    user_id = row['UserID']
    session_id = row['SessionID']
    event_type = row['EventType']
    product_id = row['ProductID'] if pd.notna(row['ProductID']) else 'N/A'
    amount = row['Amount'] if pd.notna(row['Amount']) else 'N/A'
    outcome = row['Outcome'] if pd.notna(row['Outcome']) else 'N/A'

    return (
        f"{timestamp} [INFO] - USER_ID: {user_id}, "
        f"SESSION_ID: {session_id}, "
        f"EVENT_TYPE: {event_type}, "
        f"PRODUCT_ID: {product_id}, "
        f"AMOUNT: {amount}, "
        f"OUTCOME: {outcome}"
    )


log_entries = df.apply(format_log_entry, axis=1)

with open('../data/updated/ecommerce_clickstream_transactions_reformatted.txt', 'w') as f:
    for entry in log_entries:
        f.write(entry + '\n')


Log conversion completed. Check 'data/updated/ecommerce_clickstream_transactions_reformatted.txt' for the output.


In [17]:
pd.read_csv(
    "/Users/ren/Documents/workspace/E-Commerce-Analysis/data/updated/ecommerce_clickstream_transactions_reformatted.txt").shape

(74816, 6)

In [15]:
with open("/Users/ren/Documents/workspace/E-Commerce-Analysis/data/updated/access_log_updated.txt", 'r') as file:
    line_count = sum(1 for line in file)

print(f'Số dòng trong file: {line_count}')

Số dòng trong file: 4477843
